# Практическое задание 4
# Генерация bash команды по текстовому запросу
## курс "Математические методы анализа текстов"
### ФИО: Бикметов Данил Наильевич

### Постановка задачи

В этом задании вы построите систему, выдающую пользователю последовательность утилит командной строки linux (с нужными флагами) по его текстовому запросу. Вам дан набор пар текстовый запрос - команда на выходе. 

Решение этого задания будет построено на encoder-decoder архитектуре и модели transformer.


### Библиотеки

Для этого задания вам понадобятся следующие библиотеки:
* pytorch
* transformers
* sentencepiece (bpe токенизация)
* clai utils (скачать с гитхаба отсюда https://github.com/IBM/clai/tree/nlc2cmd/utils) 


### Данные

В качестве обучающей выборке используются данные, сгенерированные автоматически по запросам с сайта stack overflow. В качестве тестовых данных используются пары запросов, размеченные асессорами.

Данные можно скачать по ссылке: https://drive.google.com/file/d/1n457AAgrMwd5VbT6mGZ_rws3g2wwdEfX/view?usp=sharing

### Метрика качества

Ваш алгоритм должен выдавать пять вариантов ответа для каждого запроса. 
Для упрощения задачи метрика качества будет учитывать утилиты и флаги ответа, но не учитывать подставленные значения. Пусть $\{ u_1, \ldots, u_T \}$, $\{ f_1, \ldots, f_T \}$ --- список утилит и множества их флагов ответа алгоритма, $\{v_1, \ldots, v_T \}$, $\{ \phi_1, \ldots, \phi_T \}$ --- список утилит и множества их флагов эталонного ответа. Если ответы отличаются по длине, они дополняются `None` утилитой. 

$$ S = \frac{1}{T} \sum_{i=1}^{T} \left(\mathbb{I}[u_i = v_i]\left( 1 + \frac{1}{2}s(f_i, \phi_i)\right) - 1\right)$$

$$ s(f, \phi) = 1 + \frac{2 |f \cap \phi| - |f \cup \phi|}{\max(|f|, |\phi|)} $$

Метрика учитывает, что предсказать правильную утилиту важнее чем правильный флаг. При этом порядок флагов не важен (однако, чтобы корректно 

## Предобработка данных (2 балла)

In [1]:
import sys
PATH_TO_CLAI_UTILS = './utils'
sys.path.append(PATH_TO_CLAI_UTILS)

In [2]:
import numpy as np
import pandas as pd

from bashlint.data_tools import bash_parser, pretty_print, cmd2template
from metric.metric_utils import compute_metric
from functools import partial

from collections import Counter
import sentencepiece as spm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

Setting bashlex grammar using file: /home/bicdan22/hw/mmot/hw4/./utils/bashlint/grammar/grammar100.txt
Bashlint grammar set up (148 utilities)



Считаем данные. В столбце `invocation` находится текстовый запрос, в столбце `cmd` находится релевантная команда.

In [3]:
train_data = pd.read_csv('data/train.csv')
train_data.head()

,invocation,cmd
0,"copy loadable kernel module ""mymodule.ko"" to t...",sudo cp mymodule.ko /lib/modules/$(uname -r)/k...
1,"display all lines containing ""ip_mroute"" in th...",cat /boot/config-`uname -r` | grep IP_MROUTE
2,display current running kernel's compile-time ...,cat /boot/config-`uname -r`
3,"find all loadable modules for current kernel, ...",find /lib/modules/`uname -r` -regex .*perf.*
4,"look for any instance of ""highmem"" in the curr...",grep “HIGHMEM” /boot/config-`uname -r`


В тестовых данных столбец `origin` отвечает за источник данных, значения `handrafted` соответствуют парам, составленными людьми, а `mined` парам, собранным автоматически.

In [5]:
test_data = pd.read_csv('data/test.csv')
test_data.head()

,invocation,cmd,origin
0,create ssh connection to specified ip from spe...,ssh user123@176.0.13.154,handcrafted
1,"search for commands containing string ""zeppeli...",history | grep zeppelin,handcrafted
2,search for location of specified file or appli...,whereis python3,handcrafted
3,grant all rights to root folder,sudo chmod 777 -R /,handcrafted
4,search in running processes for specified name,ps -aux | grep zepp,handcrafted


**Задание**. Проведите предобработку текста. Рекомендуется:
* перевести всё в нижний регистр
* удалить стоп-слова (специфичные для выборки)
* провести стемминг токенов
* удалить все символы кроме латинских букв

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

nltk.download('stopwords')

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z]', ' ', text)

    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]

    porter = PorterStemmer()
    stemmed_words = [porter.stem(word) for word in filtered_words]
    
    clean_text = ' '.join(stemmed_words)
    
    return clean_text

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/bicdan22/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
train_data['text_cleaned'] = train_data['invocation'].apply(clean_text)
test_data['text_cleaned'] = test_data['invocation'].apply(clean_text)

Для обработки кода воспользуемся функцией `cmd2template`:

In [8]:
train_data['cmd_cleaned'] = train_data['cmd'].apply(partial(cmd2template, loose_constraints=True))
test_data['cmd_cleaned'] = test_data['cmd'].apply(partial(cmd2template, loose_constraints=True))

Разделим данные на обучение и валидацию. Т.к. данных очень мало, то для валидационной выборки выделим только 100 примеров.

In [9]:
valid_data = train_data.iloc[-100:]
train_data = train_data.iloc[:-100]

In [10]:
valid_data

,invocation,cmd,text_cleaned,cmd_cleaned
9843,"searches through the root filesystem (""/"") for...",find / -name Chapter1 -type f -print,search root filesystem file name chapter print...,find Path -name Regex -type f -print
9844,"searches through the root filesystem (""/"") for...",find / -name Chapter1 -type f,search root filesystem file name chapter,find Path -name Regex -type f
9845,"searches through the root filesystem (""/"") for...",find / -name Chapter1 -type f -print,search root filesystem file name chapter,find Path -name Regex -type f -print
9846,searching for all files with the extension mp3,find / -name *.mp3,search file extens mp,find Path -name Regex
9847,set myvariable to the value of variable_name,myVariable=$(env | grep VARIABLE_NAME | grep ...,set myvari valu variabl name,env | grep Regex | grep -o -e Regex
...,...,...,...,...
9938,using exec in find command to dispaly the sear...,find . ... -exec cat {} \; -exec echo \;,use exec find command dispali search file,find Path Path -exec cat {} \; -exec echo \;
9939,verbosely create intermediate directoriy tmp a...,mkdir -pv /tmp/boostinst,verbos creat intermedi directoriy tmp requir d...,mkdir -p -v Directory
9940,view the manual page of find,man find,view manual page find,man Regex
9941,"wait 2 seconds and then print ""hello""","echo ""hello `sleep 2 &`""",wait second print hello,echo $( sleep Timespan )


**Задание**. Стандартный формат входных данных для трансформеров — BPE токены. Воспользуйтесь библиотекой sentencepiece для обучения токенайзеров для текста и кода. Используйте небольшое число токенов.

In [17]:
text_corpus = [row for row in train_data.text_cleaned]
text_corpus += [row for row in valid_data.text_cleaned]
text_corpus += [row for row in test_data.text_cleaned]
print(f'Текстовый корпус до дедупликации: {len(text_corpus)}')

# Дедупликация запросов
text_corpus = [row for row in set(text_corpus)]
print(f'Текстовый корпус после дедупликации: {len(text_corpus)}')

with open('./tokenizer/text_corpus.txt', 'w', encoding='utf-8') as f:
    for text in text_corpus:
        f.write(text + '\n')

Текстовый корпус до дедупликации: 10664
Текстовый корпус после дедупликации: 9720


In [18]:
cmd_corpus = [row for row in train_data.cmd_cleaned]
cmd_corpus += [row for row in valid_data.cmd_cleaned]
cmd_corpus += [row for row in test_data.cmd_cleaned]
print(f'Командный корпус до дедупликации: {len(cmd_corpus)}')

# Дедупликация запросов
cmd_corpus = [row for row in set(cmd_corpus)]
print(f'Командный корпус после дедупликации: {len(cmd_corpus)}')

with open('./tokenizer/cmd_corpus.txt', 'w', encoding='utf-8') as f:
    for text in cmd_corpus:
        f.write(text + '\n')

Командный корпус до дедупликации: 10664
Командный корпус после дедупликации: 5554


In [19]:
def train_tokenizer(input_file, model_prefix, vocab_size):
    spm.SentencePieceTrainer.train(
        input=input_file,
        model_prefix=model_prefix,
        vocab_size=vocab_size
    )

In [23]:
train_tokenizer("./tokenizer/text_corpus.txt", "./tokenizer/text_tokenizer", vocab_size=2500)
text_tokenizer = spm.SentencePieceProcessor(model_file="./tokenizer/text_tokenizer.model")

train_tokenizer("./tokenizer/cmd_corpus.txt", "./tokenizer/cmd_tokenizer", vocab_size=600)
cmd_tokenizer = spm.SentencePieceProcessor(model_file="./tokenizer/cmd_tokenizer.model")

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: ./tokenizer/text_corpus.txt
  input_format: 
  model_prefix: ./tokenizer/text_tokenizer
  model_type: UNIGRAM
  vocab_size: 2500
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  d

**Задание**. Задайте датасеты и лоадеры для ваших данных. Каждая последовательность должна начинаться с BOS токена и заканчиваться EOS токеном. Рекомендуется ограничить длину входных и выходных последовательностей!

In [24]:
PAD_ID = 0
BOS_ID = 1
EOS_ID = 2


MAX_TEXT_LENGTH = 256
MAX_CODE_LENGTH = 40

BATCH_SIZE = 64

In [84]:
class TextToBashDataset(Dataset):
    def __init__(self, data, text_tokenizer, cmd_tokenizer,
                 max_text_length=MAX_TEXT_LENGTH,
                 max_code_length=MAX_CODE_LENGTH):
        self.data = data
        self.text_tokenizer = text_tokenizer
        self.cmd_tokenizer = cmd_tokenizer
        self.max_text_length = max_text_length
        self.max_code_length = max_code_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]['text_cleaned']
        cmd = self.data.iloc[idx]['cmd_cleaned']

        # Токенизация и добавление BOS и EOS токенов
        text_tokens = [BOS_ID] + self.text_tokenizer.encode(text)[:self.max_text_length - 2] + [EOS_ID]
        cmd_tokens = [BOS_ID] + self.cmd_tokenizer.encode(cmd)[:self.max_code_length - 2] + [EOS_ID]

        # Дополнение нулями до максимальной длины
        text_padding = [PAD_ID] * (self.max_text_length - len(text_tokens))
        cmd_padding = [PAD_ID] * (self.max_code_length - len(cmd_tokens))

        text_tokens += text_padding
        cmd_tokens += cmd_padding

        return torch.Tensor(text_tokens).long(), torch.Tensor(cmd_tokens).long()

In [85]:
train_ds = TextToBashDataset(train_data, text_tokenizer, cmd_tokenizer)
valid_ds = TextToBashDataset(valid_data, text_tokenizer, cmd_tokenizer)

In [86]:
loaders = {
    'train': DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True),
    'valid': DataLoader(valid_ds, batch_size=BATCH_SIZE),
}

## Обучение бейзлайна (2 балла)

In [87]:
from transformers import BertConfig, BertModel, EncoderDecoderConfig, EncoderDecoderModel

**Задание.** Реализуйте модель encoder-decoder ниже. В качестве моделей энкодера и декодера рекомендуется использовать BertModel из библиотеки transformers, заданную через BertConfig. В случае декодера необходимо выставить параметры is_decoder=True и add_cross_attention=True. В качестве модели, <<сцепляющей>> энкодер и декодер, в одну архитектуру рекомендуется использовать EncoderDecoderModel.

**Обратите внимание!** EncoderDecoderModel поддерживает использование кэшированных результатов при последовательной генерации. Это пригодится при реализации beam-search ниже.

Для того, чтобы удобнее задавать модели, рекомендуется реализовать задание модели через конфиг. Ниже представлены базовые параметры, при которых модель должна работать быстро и с приемлемым качеством.

In [88]:
text_model_config = {
    'vocab': text_tokenizer.vocab_size(),
    'hidden_size': 256,
    'num_layers': 2,
    'num_attention_heads': 8,
    'intermediate_size': 256 * 4,
    'hidden_dropout_prob': 0.1,
    'pad_id': PAD_ID,
}

cmd_model_config = {
    'vocab': cmd_tokenizer.vocab_size(),
    'hidden_size': 256,
    'num_layers': 2,
    'num_attention_heads': 8,
    'intermediate_size': 256 * 4,
    'hidden_dropout_prob': 0.1,
    'pad_id': PAD_ID,
}

In [89]:
class TextToBashModel(nn.Module):
    def __init__(self, text_model_config, cmd_model_config):
        super(TextToBashModel, self).__init__()

        # Конфигурация энкодера для текста
        text_encoder_config = BertConfig(
            vocab_size=text_model_config['vocab'],
            hidden_size=text_model_config['hidden_size'],
            num_hidden_layers=text_model_config['num_layers'],
            num_attention_heads=text_model_config['num_attention_heads'],
            intermediate_size=text_model_config['intermediate_size'],
            hidden_dropout_prob=text_model_config['hidden_dropout_prob'],
            pad_token_id=text_model_config['pad_id'],
        )
        
        # Создание энкодера для текста
        self.text_encoder = BertModel(text_encoder_config)
        
        # Конфигурация декодера для команд
        cmd_decoder_config = BertConfig(
            vocab_size=cmd_model_config['vocab'],
            hidden_size=cmd_model_config['hidden_size'],
            num_hidden_layers=cmd_model_config['num_layers'],
            num_attention_heads=cmd_model_config['num_attention_heads'],
            intermediate_size=cmd_model_config['intermediate_size'],
            hidden_dropout_prob=cmd_model_config['hidden_dropout_prob'],
            pad_token_id=cmd_model_config['pad_id'],
            is_decoder=True,
            add_cross_attention=True,
        )
        
        # Создание декодера для команд
        self.cmd_decoder = BertModel(cmd_decoder_config)
        
        # Конфигурация для сцепления энкодера и декодера в одну модель
        encoder_decoder_config = EncoderDecoderConfig.from_encoder_decoder_configs(text_encoder_config, cmd_decoder_config)
        
        # Создание модели, объединяющей энкодер и декодер
        self.model = EncoderDecoderModel(encoder_decoder_config)
        
    def forward(self, input_ids, decoder_input_ids):
        # Forward pass через энкодер для текста
        text_encoder_output = self.text_encoder(input_ids=input_ids)
        
        # Forward pass через декодер для команд
        cmd_decoder_output = self.cmd_decoder(input_ids=decoder_input_ids, encoder_hidden_states=text_encoder_output.last_hidden_state)
        
        # Forward pass через общую модель
        model_output = self.model(input_ids=input_ids, decoder_input_ids=decoder_input_ids)
        
        return model_output.logits

**Задание**. Обучите вашу модель ниже.

Рекомендуется:
* в качестве лосса использовать стандартную кросс-энтропию, не забывайте игнорировать PAD токены
* использовать Adam для оптимизации
* не использовать scheduler для бейзлайна (модель легко переобучается с ним)
* использовать early stopping по валидационному лоссу

In [90]:
import torch.optim as optim
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Разделение данных на обучающую и валидационную выборки
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)

# Создание экземпляра модели
model = TextToBashModel(text_model_config, cmd_model_config)

# Определение устройства (CPU или GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Критерий (лосс) - кросс-энтропия
criterion = nn.CrossEntropyLoss(ignore_index=PAD_ID)

# Оптимизатор - Adam
optimizer = optim.Adam(model.parameters())

# Количество эпох
num_epochs = 10

# Early stopping
best_val_loss = float('inf')
patience = 3
counter = 0

# Цикл обучения
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    
    # Итерация по обучающему датасету
    for batch in tqdm(loaders['train']):
        input_ids = batch[0].to(device)
        decoder_input_ids = batch[1].to(device)
        
        # Forward pass
        logits = model(input_ids, decoder_input_ids)
        
        # Подсчет лосса
        loss = criterion(logits.view(-1, logits.size(-1)), decoder_input_ids.view(-1))
        
        # Backward pass и обновление весов
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    # Вычисление среднего лосса для текущей эпохи
    average_loss = total_loss / len(loaders['train'])
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {average_loss:.4f}")
    
    # Валидация модели
    model.eval()
    val_loss = 0
    
    with torch.no_grad():
        # Итерация по валидационному датасету
        for batch in tqdm(loaders['valid']):
            input_ids = batch[0].to(device)
            decoder_input_ids = batch[1].to(device)
            
            # Forward pass
            logits = model(input_ids, decoder_input_ids)
            
            # Подсчет лосса
            loss = criterion(logits.view(-1, logits.size(-1)), decoder_input_ids.view(-1))
            
            val_loss += loss.item()
    
    # Вычисление среднего валидационного лосса
    average_val_loss = val_loss / len(loaders['valid'])
    print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {average_val_loss:.4f}")
    
    # Проверка Early stopping
    if average_val_loss < best_val_loss:
        best_val_loss = average_val_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping")
            break


100%|██████████| 82/82 [02:47<00:00,  2.04s/it]


Epoch 1/10, Train Loss: 0.8263


100%|██████████| 2/2 [00:00<00:00,  2.14it/s]


Epoch 1/10, Validation Loss: 0.1671


100%|██████████| 82/82 [02:39<00:00,  1.95s/it]


...

Epoch 9/10, Validation Loss: 0.0295


100%|██████████| 82/82 [02:40<00:00,  1.96s/it]


Epoch 10/10, Train Loss: 0.0010


100%|██████████| 2/2 [00:00<00:00,  2.15it/s]

Epoch 10/10, Validation Loss: 0.0292


## Генерация команд (2 балла)

**Задание**. Реализуйте алгоритм beam-search в классе BeamSearchGenerator ниже. Ваша реализация должна поддерживать задание температуры софтмакса. Выходы модели, полученные на предыдущих итерациях, необходимо кэшировать для повышения скорости алгоритма. Вместо подсчёта произведения любых вероятностей необходимо считать сумму их логарифмов.

Алгоритм должен возвращать список пар из получившихся выходных последовательностей и логарифмов их вероятностей. 

In [117]:
class BeamSearchGenerator:
    def __init__(
            self, pad_id, eos_id, bos_id,
            max_length=20, beam_width=5, temperature=1,
            device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    ):
        """
        Parameters
        ----------
        pad_id : int
        eos_id : int
        bos_id : int
        max_length : int
            Maximum length of output sequence
        beam_width : int
            Width of the beam
        temperature : float
            Softmax temperature
        device : torch.device
            Your model device
        """
        self.pad_id = pad_id
        self.eos_id = eos_id
        self.bos_id = bos_id
        
        self.max_length = max_length
        self.beam_width = beam_width
        self.temperature = temperature
        
        self.device = device
        
    def get_result(self, model, input_text_tokens):
        """
        Parameters
        ----------
        model : TextToBashModel
        input_text_tokens : torch.tensor
            One object input tensor
        """
        # Запуск энкодера на входном тексте
        encoder_output = model.text_encoder(input_ids=input_text_tokens.unsqueeze(0).to(self.device))
        
        # Начальное состояние beam search
        beam_search_state = [{
            'sequence': [self.bos_id],
            'log_prob': 0.0,
            'length': 1,
            'last_hidden_state': encoder_output.last_hidden_state,
        }]
        
        # Итерации по длине последовательности
        for length in range(1, self.max_length + 1):
            # Список для хранения нового состояния beam search
            new_beam_search_state = []
            
            # Итерации по текущему состоянию beam search
            for state in beam_search_state:
                # Если последовательность достигла максимальной длины, добавляем в результат
                if state['sequence'][-1] == self.eos_id or state['length'] == self.max_length:
                    new_beam_search_state.append(state)
                else:
                    # Подготавливаем вход для декодера
                    decoder_input_ids = torch.tensor([state['sequence']], device=self.device)
                    
                    # Forward pass декодера
                    logits = model.cmd_decoder(input_ids=decoder_input_ids, encoder_hidden_states=state['last_hidden_state'])[0]
                    logits = logits[:, -1, :] / self.temperature
                    
                    # Применение softmax для получения вероятностей
                    probabilities = nn.functional.softmax(logits, dim=-1)
                    
                    # Получение top-k индексов
                    top_k_probs, top_k_indices = torch.topk(probabilities, self.beam_width, dim=-1)
                    
                    # Расчет нового состояния beam search для каждого top-k индекса
                    for k in range(self.beam_width):
                        new_sequence = state['sequence'] + [top_k_indices[0, k].item()]
                        new_log_prob = state['log_prob'] + top_k_probs[0, k].log().item()
                        new_length = state['length'] + 1
                        
                        # Кэширование результатов предыдущей итерации для ускорения
                        new_last_hidden_state = state['last_hidden_state']
                        if k > 0:
                            new_last_hidden_state = torch.cat([new_last_hidden_state, state['last_hidden_state']], dim=0)
                        
                        new_beam_search_state.append({
                            'sequence': new_sequence,
                            'log_prob': new_log_prob,
                            'length': new_length,
                            'last_hidden_state': new_last_hidden_state,
                        })
            
            # Сортировка нового состояния beam search по логарифму вероятности
            beam_search_state = sorted(new_beam_search_state, key=lambda x: x['log_prob'], reverse=True)[:self.beam_width]
        
        # Отбор лучших результатов
        best_results = sorted(beam_search_state, key=lambda x: x['log_prob'], reverse=True)
        
        # Возвращение списка пар результатов
        return [(result['sequence'], result['log_prob']) for result in best_results]

Протестируйте на нескольких примерах работу вашего алгоритма. Если всё реализовано правильно, то как минимум на трёх примерах из 5 всё должно работать правильно.

In [118]:
beam_search_engine = BeamSearchGenerator(
    pad_id=PAD_ID, eos_id=EOS_ID, bos_id=BOS_ID,
    max_length=MAX_CODE_LENGTH, beam_width=5,
    temperature=1, device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
)

In [119]:
with torch.no_grad():
    for i in range(5):
        print()
        print('text:', valid_data.invocation.iloc[i])
        print('true:', valid_data.cmd.iloc[i])
        print('true cleaned:', valid_data.cmd_cleaned.iloc[i])

        src = valid_ds[i][0]
        pred = beam_search_engine.get_result(model, src)
        
        scores = []
        for x, proba in pred:
            pred_cmd = cmd_tokenizer.decode(list(map(int, x)))
            score = compute_metric(pred_cmd, 1, valid_data.cmd.iloc[i])
            scores.append(score)
            print(pred_cmd, proba)
        print(max(scores))


text: searches through the root filesystem ("/") for the file named chapter1, and prints the location
true: find / -name Chapter1 -type f -print
true cleaned: find Path -name Regex -type f -print
prune -2prune bas bind -2amensize ifconfig chgrp ' "%prunename DateTime -- true read bindand \o Path set ifconfig-%scp3@ -2name3@ -2"scp Permission -120.37553215026855
prune -2prune bas bind -2amensize ifconfig chgrp ' "%prunename DateTime -- true read bindand \o Path set ifconfig-%scp3@ -2name3@ -2"scpS -120.6160933971405
prune -2prune bas bind -2amensize ifconfig chgrp ' "%prunename DateTime -- true read bindand \o Path set ifconfig-%scp3@ -2name3@ -2"okor -120.67719459533691
prune -2prune bas bind -2amensize ifconfig chgrp ' "%prunename DateTime -- true read bindand \o Path set ifconfig-%scp3@ -2name3@ok"scp Permission -120.80940866470337
prune -2prune bas bind -2amensize ifconfig chgrp ' "%prunename DateTime -- true read bindand \o Path set ifconfig-%scp3@ -2name3@<"scp Permission -120.84

**Задание**. Дополните функцию для подсчёта качества. Посчитайте качество вашей модели на валидационном и тестовых датасетов.

In [122]:
def compute_all_scores(model, df, beam_engine):
    all_scores = []

    for i, (text, target_cmd) in enumerate(zip(df.text_cleaned.values, df.cmd.values)):
        input_tokens = [text_tokenizer.bos_id()] + text_tokenizer.encode(text)
        predictions = beam_engine.get_result(model, input_tokens)
        
        # get only 5 top results
        predictions = predictions[:5]
        object_scores = []
        for output_tokens, proba in predictions:
            output_cmd = cmd_tokenizer.decode(list(map(int, output_tokens)))
            score = compute_metric(output_cmd, 1, target_cmd)
            object_scores.append(score)
        
        all_scores.append(max(object_scores))
    return all_scores

Ваша цель при помощи подбора параметров модели и генерации получить средний скор на валидации >= 0.2, скор `handcrafted` части теста >= 0.1. На `mined` части датасета скор может быть низкий, т.к. некоторых команд из датасета нет в обучении.

**Обратите внимание.** Так как датасет для обучения не очень большой, а данные достаточно нестабильные, подбор параметров может очень сильно влиять на модель. Некоторые полезные советы:
* Отслеживайте качество модели после каждой эпохи, не забывайте про early stopping
* Вы можете сразу приступить к следующей части. Побитие скора в этой части задания при помощи трюков из бонусной части считается валидным.

In [ ]:
## YOUR CODE HERE ##

## Улучшение модели (4 балла)

Вы реализовали бейзлайн, пришло время улучшить качество модели. Т.к. это последнее задание, мы не будем предлагать конкретные шаги, а только дадим несколько советов.

1. Большой источник информации о работе командной строке — её документация, man. Один из способов улучшения модели - использование мана для генерации новых примеров. Структурированный ман можно найти по ссылке https://github.com/IBM/clai/blob/nlc2cmd/docs/manpage-data.md.
2. Ещё один способ улучшить модель, разделить предсказание утилит и флагов. Т.к. задача предсказания утилит более важная, вы можете натренировать модель, которая предсказывает последовательность утилит, а затем к каждой утилите генерировать флаги.
3. Можно аугментировать данные, чтобы увеличить выборку.
4. Можно в качество входа подавать не только текстовый запрос, но и описание из мана. Т.к. всё описание достаточно большое, нужно сделать дополнительную модель, которая будет выбирать команды, для которых нужно вытащить описание.
5. Найти дополнительные данные, улучшающие обучение
6. Как всегда можно просто сделать больше слоёв, увеличить размер скрытого слоя и т.д.

От вас ожидается скор на валидации >= 0.25, `mined` >= 0, `handrafted` >= 0.15.

In [ ]:
## YOUR CODE HERE ##

## Бонусные баллы (до 3 баллов)

При существенном улучшении качества будут назначаться бонусные баллы. На тестовых датасетах реально выбить качество >= 0.3 на каждом, но усилий потребуется немало...